In [11]:
# from google.colab import drive
from pathlib import Path

# drive.mount("/content/drive")
# project_path = Path("/content/drive/MyDrive/NLP/MultiModalEmotionRecognition")
import sys, os
from pathlib import Path

project_path = Path(os.path.dirname(os.getcwd()))
sys.path.append(str(project_path))

In [5]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
import os, cv2, torch, ast
import pandas as pd
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
from tqdm import trange
from tqdm import tqdm


class MSCTDDataSet(Dataset):
    """MSCTD dataset."""

    def __init__(
        self,
        base_path="data/",
        split="train",
        data_size=None,
        load=False,
        device="cuda",
    ):
        """
        Args:
            base_path (str or path): path to data folder
            split (str): dev, train, test
        """
        if isinstance(base_path, str):
            base_path = Path(base_path)
        self.base_path = base_path
        self.load_path = base_path / "saved_features"
        self.split = split
        self.sentiment_file_path = base_path / f"sentiment_{split}.txt"
        self.image_dir = base_path / "images" / split

        self.data_size = data_size
        self.load = load
        self.device = device

        self.sentiments = None
        self.indexes = None
        self.face_embeddings = None
        self.load_data()

    def load_data(self):
        if self.load:
            try:
                indexes = torch.load(self.load_path / f"real_indexes_{self.split}.pt")
                sentiments = torch.load(self.load_path / f"sentiments_{self.split}.pt")
                face_embeddings = torch.load(
                    self.load_path / f"face_embeddings_{self.split}.pt"
                )

                assert (
                    face_embeddings.shape[0] == indexes.shape[0]
                ), "ERROR: face and index list are not the same size in loading"
                assert (
                    indexes.shape[0] == sentiments.shape[0]
                ), "ERROR: index and sentiment list are not the same size in loading"

            except Exception as e:
                print(e)
                print(
                    "Warning: passed load=True but not embedding file was located. Not loading"
                )
                if str(e).startswith("ERROR"):
                    raise e

        if self.data_size:
            indexes = indexes[: self.data_size]
            sentiments = sentiments[: self.data_size]
            if not face_embeddings is None:
                face_embeddings = face_embeddings[: self.data_size, :]

        self.sentiments = sentiments
        self.indexes = indexes
        self.face_embeddings = face_embeddings

    def __len__(self):
        if self.load:
            return self.face_embeddings.shape[0]

    def get_face_embeddings(self, index):
        return self.face_embeddings[index]

    def get_sentiment(self, index):
        return self.sentiments[index]

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        try:
            face_embedding = self.get_face_embeddings(index)
        except Exception as e:
            print(f"error for split:{self.split} index: {index}")
            print(e)

        sentiment = self.get_sentiment(index)
        sample = {
            "index": self.indexes[index],
            "face_embedding": face_embedding,
            "sentiment": sentiment,
        }
        return sample

In [2]:
class MSCTDDataLoader:
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)


def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    if isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    if isinstance(data, str):
        return data
    return data.to(device)

In [13]:
import torch
from torch import nn


class SimpleDenseNetwork(nn.Module):
    def __init__(self, n_classes, embedding_dimension):
        super(SimpleDenseNetwork, self).__init__()

        self.n_classes = n_classes
        self.embedding_dimension = embedding_dimension

        self.fc = nn.Sequential(
            nn.Linear(
                in_features=self.embedding_dimension,
                out_features=512,
            ),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=128, out_features=3),
        )

    def forward(self, input_batch):
        x = input_batch
        x = self.fc(x)
        output_batch = x

        return output_batch

In [18]:
BATCH_SIZE = 32
num_workers = 1
EPOCHS = 10
embedding_dimension = 1280

learning_rate = 0.001
momentum = 0.001
data_size = None

In [14]:
from torch.utils.data import DataLoader

val_dataset = MSCTDDataSet(project_path / "data", "val", data_size=data_size, load=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
val_dataloader = MSCTDDataLoader(val_dataloader, device)

train_dataset = MSCTDDataSet(project_path / "data", "train", load=True)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
train_dataloader = MSCTDDataLoader(train_dataloader, device)

# test_dataset = MSCTDDataSet(project_path / "data", "test", load=True)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
# test_dataloader = MSCTDDataLoader(test_dataloader, device)

In [23]:
import evaluate

accuracy = evaluate.load("accuracy")


def validate(model, dataloader, loss_fn):
    running_loss = 0.0
    last_loss = 0.0

    for data_pair_index, batch in enumerate(dataloader):
        face_embedding = batch["face_embedding"]
        labels = batch["sentiment"]
        logits = model(face_embedding)
        accuracy.add_batch(predictions=logits.argmax(dim=1), references=labels)
        loss = loss_fn(logits, labels)
        running_loss += loss.item()
    print(accuracy.compute())

In [21]:
import torch.optim as optim
from datetime import datetime


def train_epoch(epoch_index, model, dataloader, loss_fn, optimizer):
    running_loss = 0.0
    # last_loss = 0.0

    for data_pair_index, batch in enumerate(dataloader):
        face_embedding = batch["face_embedding"]
        labels = batch["sentiment"]
        optimizer.zero_grad()
        outputs = model(face_embedding)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch loss: ", running_loss)


def train_model(model, epochs, train_dataloader, val_dataloader):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(epochs):
        print("--------------epoch: ", epoch, "-------------")
        model.train()
        train_epoch(epoch, model, train_dataloader, loss_fn, optimizer)
        model.eval()
        validate(model, val_dataloader, loss_fn)

In [17]:
model = SimpleDenseNetwork(n_classes=3, embedding_dimension=embedding_dimension).to(
    device=device
)

In [24]:
train_model(model, EPOCHS, train_dataloader, val_dataloader)

--------------epoch:  0 -------------
Epoch loss:  434.9349400997162
{'accuracy': 0.3662267546490702}
--------------epoch:  1 -------------
Epoch loss:  433.03176885843277
{'accuracy': 0.3662267546490702}
--------------epoch:  2 -------------
Epoch loss:  431.8867508172989
{'accuracy': 0.36682663467306537}
--------------epoch:  3 -------------
Epoch loss:  429.58835250139236
{'accuracy': 0.368626274745051}
--------------epoch:  4 -------------
Epoch loss:  426.0436946749687
{'accuracy': 0.3728254349130174}
--------------epoch:  5 -------------
Epoch loss:  423.6378684043884
{'accuracy': 0.3707258548290342}
--------------epoch:  6 -------------
Epoch loss:  418.87018913030624
{'accuracy': 0.3698260347930414}
--------------epoch:  7 -------------
Epoch loss:  413.3151898384094
{'accuracy': 0.371625674865027}
--------------epoch:  8 -------------
Epoch loss:  405.90360221266747
{'accuracy': 0.371625674865027}
--------------epoch:  9 -------------
Epoch loss:  396.8242737650871
{'accuracy'